<a href="https://colab.research.google.com/github/mkovac03/gee_tools/blob/master/ee_export_to_harddrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Status:
pull ndvi+QA: working
mosaic ndvi+QA: working 
- need to figure out whether I want to manually move files from google drive each time or just write them directly to local drive

next steps: 
- Figure out what I need to project to be able to clip land cover correctly
- write code to create .txt file


In [2]:
!pip install earthengine-api
!pip install geemap
!pip install earthpy
!conda install osgeo
!pip install pyproj
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import ee
import numpy as np
import pandas as pd
import geemap
import matplotlib.pyplot as plt
import os
from osgeo import gdal
from osgeo import osr

In [ ]:
# ee.Authenticate()
ee.Initialize()

path = '/change/to/path'
try:
    os.mkdir(path)
except:
    print('Folder already exists.')
    pass
    
os.chdir(path)
# Get the current working directory
cwd = os.getcwd()
# Print the current working directory
print("Current working directory: {0}".format(cwd))

# changeables

In [ ]:
# AREA OF INTEREST

# IF I WANT ALL OF SA:
#countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
#aoi = ee.Feature(countries.filter(ee.Filter.inList('country_na',
# ['South Africa'])).union().first()).geometry()

#IF I WANT AN AREA FOR TESTING (I DO)
aoi = ee.Geometry.Polygon(
    [[[25.9, -28.5], [26.6, -28.5], [26.6, -27.9], [25.9, -27.9]]], None, False
)


# DATES
start_date = '2003-01-01'
end_date = '2004-12-31'

# DIRECTORIES - if I want to export to local drive instead of google drive 
# --> https://tutorials.geemap.org/AssetManagement/export_data/
#out_dir = 


# pull NDVI by grid

In [ ]:
Map = geemap.Map()

Map.addLayer(aoi)
# Zoom to regions of interest
Map.centerObject(aoi, 10)

dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .select(['NDVI', 'DetailedQA'])
ndviCol = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .select(['NDVI'])
qaCol = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .select(['DetailedQA'])

ndviVis = {
    'bands': 'NDVI',
    'min': 0.0,
    'max': 8000.0,
    'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
Map.addLayer(dataset.first(), ndviVis, 'NDVI')
Map.addLayer(aoi, {}, 'aoi')
Map

for testing:

In [ ]:
# global UTM grid
utm = ee.FeatureCollection('users/gmkovacs/World_UTM_Grid')
Map.addLayer(utm, {}, 'UTM grids')

In [ ]:
# transfor the collection into a multiband image
ndviSeries = ndviCol.toBands()
# get the image names
ndvi_names = ndviSeries.bandNames().getInfo()
#extract image dates
ndvi_dates = [n[:-5].replace('_', '.') for n in ndvi_names]
print(ndvi_dates)

df = pd.DataFrame(ndvi_dates)
# save image dates
df.to_csv('ndvi_dates.csv', mode='a', index=False, header=True)

In [ ]:
# get the number of images
n_img = ndviCol.size().getInfo()
# get list of images
imgList = dataset.toList(n_img)
print(f'number of images: {n_img}')

In [ ]:
from pyproj import CRS
# get utm zone for the aoi
utm_zone = utm.filterBounds(aoi).first().get('ZONE').getInfo()
# compute the crs
crs = CRS.from_dict({'proj': 'utm', 'zone': utm_zone, 'south': False})
# set noData value to -20000 and reproject
img = ndviSeries.unmask(-20000).reproject(f'EPSG:{crs.to_authority()[1]}', None, 250)
# Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
band_arrs = img.sampleRectangle(region=aoi)

# get the first image
band_arr_ndvi = band_arrs.get(ndvi_names[0])
np_arr_ndvi0 = np.array(band_arr_ndvi.getInfo())
print(np_arr_ndvi0.shape)

# Get individual band arrays.
arrs = np.array([np_arr_ndvi0])
for i in ndvi_names[1:]:
    band_arr_ndvi = band_arrs.get(i)
    # Transfer the arrays from server to client and cast as np array.
    np_arr_ndvi = np.array([band_arr_ndvi.getInfo()])
    arrs = np.append(arrs, np_arr_ndvi, axis = 0)
print(arrs.shape)

# Stack the individual images to make a 3-D array.
np_stack_img = np.stack(arrs) 

In [ ]:
import earthpy.plot as ep
from matplotlib.colors import ListedColormap

# plot firt images
ep.plot_bands(np_stack_img[0], cmap='RdYlGn', title = f"{imgList.get(0).getInfo()['id']}")
plt.show()

In [ ]:
## save multiband NDVI with GDAL

nbands, height, width = np_stack_img.shape
# get the lat lon and add the ndvi
latlon = img.pixelCoordinates(f'EPSG:{crs.to_authority()[1]}')
# apply reducer to list
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=aoi,
    maxPixels=3e8,
    scale=250)
lats = np.array((ee.Array(latlon.get('y')).getInfo()))
lons = np.array((ee.Array(latlon.get('x')).getInfo()))
# get the unique coordinates
uniqueLats = np.unique(lats)
uniqueLons = np.unique(lons)
# get number of columns and rows from coordinates
ncols = len(uniqueLons)    
nrows = len(uniqueLats)
# determine pixelsizes
ys = uniqueLats[1] - uniqueLats[0] 
xs = uniqueLons[1] - uniqueLons[0]
# set the coordinate system
target = osr.SpatialReference()
target.ImportFromEPSG(int(crs.to_authority()[1]))
transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
datas = np_stack_img
driver = gdal.GetDriverByName("GTiff")
tods = driver.Create(f"{imgList.get(0).getInfo()['id'].replace('/', '_')}_{imgList.get(-1).getInfo()['id'].replace('/', '_')[-10:]}_aoi_NDVI.tif", width, height, nbands, eType=gdal.GDT_Int16)
tods.WriteRaster(0,0,width,height,datas,band_list=list(range(1, nbands+1)))
tods.SetGeoTransform(transform)
tods.SetProjection(target.ExportToWkt())
tods.FlushCache()

In [ ]:
# Save multiband QA with GDAl

qaSeries = qaCol.toBands()
qa_names = qaSeries.bandNames().getInfo()

img = qaSeries.reproject(f'EPSG:{crs.to_authority()[1]}', None, 250)
# Get 2-d pixel array for AOI - returns feature with 2-D pixel array as property per band.
band_arrs = img.sampleRectangle(region=aoi)

band_arr_qa = band_arrs.get(qa_names[0])
np_arr_qa0 = np.array(band_arr_qa.getInfo())

# Get individual band arrays.
arrs = np.array([np_arr_qa0])
for i in qa_names[1:]:
    band_arr_qa = band_arrs.get(i)
    # Transfer the arrays from server to client and cast as np array.
    np_arr_qa = np.array([band_arr_qa.getInfo()])
    arrs = np.append(arrs, np_arr_qa, axis = 0)
# Stack the individual images to make a 3-D array.
np_stack_img = np.stack(arrs) 
# plot firt images
ep.plot_bands(np_stack_img[0], cmap='RdYlGn', title = f"{imgList.get(0).getInfo()['id']}")
plt.show()

In [ ]:
nbands, height, width = np_stack_img.shape
# get the lat lon and add the ndvi
latlon = img.pixelCoordinates(f'EPSG:{crs.to_authority()[1]}')
# apply reducer to list
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=aoi,
    maxPixels=3e8,
    scale=250)
lats = np.array((ee.Array(latlon.get('y')).getInfo()))
lons = np.array((ee.Array(latlon.get('x')).getInfo()))
# get the unique coordinates
uniqueLats = np.unique(lats)
uniqueLons = np.unique(lons)
# get number of columns and rows from coordinates
ncols = len(uniqueLons)    
nrows = len(uniqueLats)
# determine pixelsizes
ys = uniqueLats[1] - uniqueLats[0] 
xs = uniqueLons[1] - uniqueLons[0]
# set the coordinate system
target = osr.SpatialReference()
target.ImportFromEPSG(int(crs.to_authority()[1]))
transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
datas = np_stack_img
driver = gdal.GetDriverByName("GTiff")
tods = driver.Create(f"{imgList.get(0).getInfo()['id'].replace('/', '_')}_{imgList.get(-1).getInfo()['id'].replace('/', '_')[-10:]}_aoi_QA.tif", width, height, nbands, eType=gdal.GDT_Int16)
tods.WriteRaster(0,0,width,height,datas,band_list=list(range(1, nbands+1)))
tods.SetGeoTransform(transform)
tods.SetProjection(target.ExportToWkt())
tods.FlushCache()

In [ ]:
# # Run this to save the images separately with the QA band

# import time

# for i in range(n_img):
#     utm_zone = utm.filterBounds(aoi).first().get('ZONE').getInfo()
#     crs = CRS.from_dict({'proj': 'utm', 'zone': utm_zone, 'south': False})
#     image = ee.Image(imgList.get(i)).unmask(-20000).reproject(f'EPSG:{crs.to_authority()[1]}', None, 250)
#     bands = image.bandNames().getInfo()
#     band_arrs = image.sampleRectangle(region=aoi)
#     np_arrs = []
#     for band in bands:
#         np_arr = np.array(band_arrs.get(band).getInfo())
#         np_arrs.append(np_arr)
#     np_stack_img = np.stack((np_arrs))
#     nbands, height, width = np_stack_img.shape
#     # get the lat lon and add the ndvi
#     latlon = image.pixelCoordinates(f'EPSG:{crs.to_authority()[1]}')
#     # apply reducer to list
#     latlon = latlon.reduceRegion(
#         reducer=ee.Reducer.toList(),
#         geometry=aoi,
#         maxPixels=3e8,
#         scale=250)
#     lats = np.array((ee.Array(latlon.get('y')).getInfo()))
#     lons = np.array((ee.Array(latlon.get('x')).getInfo()))
#     # get the unique coordinates
#     uniqueLats = np.unique(lats)
#     uniqueLons = np.unique(lons)
#     # get number of columns and rows from coordinates
#     ncols = len(uniqueLons)    
#     nrows = len(uniqueLats)
#     # determine pixelsizes
#     ys = uniqueLats[1] - uniqueLats[0] 
#     xs = uniqueLons[1] - uniqueLons[0]
#     # set the coordinate system
#     target = osr.SpatialReference()
#     target.ImportFromEPSG(int(crs.to_authority()[1]))
#     transform = (np.min(uniqueLons),xs,0,np.max(uniqueLats),0,-ys)
#     datas = np_stack_img
#     driver = gdal.GetDriverByName("GTiff")
#     tods = driver.Create(f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_NDVI_QA_aoi.tif", width, height, nbands, eType=gdal.GDT_Int16)
#     tods.WriteRaster(0,0,width,height,datas,band_list=list(range(1, nbands+1)))
#     tods.SetGeoTransform(transform)
#     tods.SetProjection(target.ExportToWkt())
#     tods.FlushCache()
#     print(f"Exporting {imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_ND_aoi.tif to hardrive...")
# print('Export completed.')

#### EXPORT TO DRIVE-VERSION

import time
for i in range(n_img):
    for n in range(n_feat):
        grid_n = ee.Feature(featList.get(n)).buffer(distance = 500).bounds()
        task = ee.batch.Export.image.toDrive(
            image = ee.Image(imgList.get(i)).clip(grid_n),
            description= f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_ND_{n}",
            region= grid_n.geometry(),
            scale= 250,
            maxPixels= 10e12,
            # change this to the desired folder name on Google Drive
            folder= f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_ND",
            crs='EPSG:4326',
            fileFormat='GeoTIFF')
        task.start()
        print(f"Exporting {imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_{n} to Drive...")
        while task.active():
            time.sleep(30)
        if task.status()['state'] != 'COMPLETED':
            print('Error with export.')
        else:
            print('Export completed.')

# Pull Quality Assessment by grid

In [ ]:
Map = geemap.Map()

Map.addLayer(aoi)
# Zoom to regions of interest
Map.centerObject(aoi, 6)

dataset = ee.ImageCollection('MODIS/061/MOD13Q1') \
                  .filter(ee.Filter.date(start_date, end_date)) \
                  .map(lambda image: image.clip(aoi)) \
                  .select('DetailedQA')

ndviVis = {
  'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
Map.addLayer(dataset.first(), ndviVis, 'NDVI')

grids = dataset.geometry().coveringGrid('EPSG:4326', 100000)
Map.addLayer(grids, {}, 'grid')
Map



In [ ]:
n_img = dataset.size().getInfo()
print(f'number of images: {n_img}')

n_feat = grids.size().getInfo()
print(f'number of grids: {n_feat}')

In [ ]:
imgList = dataset.toList(n_img)
featList = grids.toList(n_feat)

In [ ]:
import time
for i in range(n_img):
    for n in range(n_feat):
        grid_n = ee.Feature(featList.get(n)).buffer(distance = 500).bounds()
        task = ee.batch.Export.image.toDrive(
            image = ee.Image(imgList.get(i)).clip(grid_n),
            description= f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_{n}",
            region= grid_n.geometry(),
            scale= 250,
            maxPixels= 10e12,
            # change this to the desired folder name on Google Drive
            folder= f"{imgList.get(i).getInfo()['id'].replace('/', '_')}_QA",
            crs='EPSG:4326',
            fileFormat='GeoTIFF')
        task.start()
        print(f"Exporting {imgList.get(i).getInfo()['id'].replace('/', '_')}_sa_QA_{n} to Drive...")
        while task.active():
            time.sleep(30)
        if task.status()['state'] != 'COMPLETED':
            print('Error with export.')
        else:
            print('Export completed.')

## Mosaic NDVI and QA
using rasterio and glob https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html
but make it iterate over folders

Before doing this, tiles must be moved from google drive. This might be possible to change, but I'm currently (June 16th) not sure)

In [ ]:
import os, glob
import rasterio
from rasterio.merge import merge 
from rasterio.plot import show

### Mosaic - WORKING

In [ ]:
rootdir = "C:/PythonProgrammingTemp/NDVItest/"
for subdir, dirs, files in os.walk(rootdir): 
    for dir in dirs:
        dirpath = os.path.join(rootdir,dir)
        filename = dirpath[-16:]
        # skift evt dirpath ud med rootdir herunder - så alle .tif kommer til at ligge i en enkelt mappe
        out_fp = f"{dirpath}/{filename}.tif"
        search_criteria = "*.tif"
        q = os.path.join(dirpath, search_criteria)
        print(q)

        dem_fps = glob.glob(q)
        print(dem_fps)

        print(out_fp)

        src_files_to_mosaic = []
        for fp in dem_fps:
            src = rasterio.open(fp)
            src_files_to_mosaic.append(src)

        mosaic, out_trans = merge(src_files_to_mosaic)
        show(mosaic, cmap='terrain')
        
        out_meta = src.meta.copy()
        # Update the metadata
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         "crs": "+proj=utm +zone=35 +ellps=GRS80 +units=m +no_defs "
                            }
                       )
        with rasterio.open(out_fp, "w", **out_meta) as dest:
            dest.write(mosaic)


#### for single file

dirpath = "C:/PythonProgrammingTemp/NDVItest/MODIS_061_MOD13Q1_2003_01_01_sa_QA/"
out_fp = f"{dirpath}\_full.tif"
search_criteria = "*.tif"
q = os.path.join(dirpath, search_criteria)
print(q)

dem_fps = glob.glob(q)
print(dem_fps)

print(out_fp)

src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

mosaic, out_trans = merge(src_files_to_mosaic)
show(mosaic, cmap='terrain')


## Create list of files for NDVI and QA
Probably: iterate through all folders to find the .tif file for each year, append to array, write to .txt

## Cut Land Cover to NDVI-area






Import LC data
project to the same coordinate system as the NDVI/QA
clip to NDVI dataset
resize raster to NDVI pixels 
reclass 



in_tif = 'C:\Users\KKBR\OneDrive - Københavns Universitet\Uni\Speciale\SA_egis_data\NLC_2020.tif'